<a href="https://colab.research.google.com/github/Yannethtj/PruebasSaberProInteligenciaArtificial/blob/main/99_modelo_solucion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!kaggle
!chmod 600 ./kaggle.json

usage: kaggle [-h] [-v] [-W] {competitions,c,datasets,d,kernels,k,models,m,files,f,config} ...
kaggle: error: the following arguments are required: command


In [ ]:
!kaggle competitions download -c udea-ai4eng-20242

 70% 14.0M/20.1M [00:00<00:00, 54.1MB/s]
100% 20.1M/20.1M [00:00<00:00, 60.8MB/s]


In [ ]:
!unzip udea-ai4eng-20242.zip


Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [52]:

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt




In [29]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head().T

,0,1,2,3,4
ID,904256,645256,308367,470353,989032
PERIODO,20212,20212,20203,20195,20212
ESTU_PRGM_ACADEMICO,ENFERMERIA,DERECHO,MERCADEO Y PUBLICIDAD,ADMINISTRACION DE EMPRESAS,PSICOLOGIA
ESTU_PRGM_DEPARTAMENTO,BOGOTÁ,ATLANTICO,BOGOTÁ,SANTANDER,ANTIOQUIA
ESTU_VALORMATRICULAUNIVERSIDAD,Entre 5.5 millones y menos de 7 millones,Entre 2.5 millones y menos de 4 millones,Entre 2.5 millones y menos de 4 millones,Entre 4 millones y menos de 5.5 millones,Entre 2.5 millones y menos de 4 millones
ESTU_HORASSEMANATRABAJA,Menos de 10 horas,0,Más de 30 horas,0,Entre 21 y 30 horas
FAMI_ESTRATOVIVIENDA,Estrato 3,Estrato 3,Estrato 3,Estrato 4,Estrato 3
FAMI_TIENEINTERNET,Si,No,Si,Si,Si
FAMI_EDUCACIONPADRE,Técnica o tecnológica incompleta,Técnica o tecnológica completa,Secundaria (Bachillerato) completa,No sabe,Primaria completa
FAMI_EDUCACIONMADRE,Postgrado,Técnica o tecnológica incompleta,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,Primaria completa


In [53]:
def limpiar_y_preprocesar(data):

  data['PERIODO'] = data['PERIODO'].astype(str).apply(lambda text: text[:4])
  def normalize_text(text: str):
    import unicodedata
    import re
    if not pd.isna(text):
        # Normaliza el texto eliminando acentos
        normalized = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
        # Elimina caracteres no deseados, incluidos "¿" y "?"
        return re.sub(r'[¿?]', '', normalized)
    return text

  def group_values_by_frequency(data: pd.DataFrame, col: str, thresholds: list[float] = [0.05, 0.2]):
    """
    Agrupa los valores de una columna en función de su frecuencia relativa.
    Los valores se agrupan en categorías basadas en una lista de umbrales.

    Args:
        data (pd.DataFrame): DataFrame de entrada.
        col (str): Nombre de la columna a procesar.
        thresholds (list[float]): Lista de umbrales para definir los grupos.

    Returns:
        pd.Series: Serie con los valores agrupados.
    """
    porcentaje = data[col].value_counts(normalize=True)

    def asignar_grupo(value):
        freq = porcentaje[value]
        for i, t in enumerate(thresholds):
            if freq < t:
                return f'GRUPO_{i+1}'
        return 'GRUPO_ALTO'

    return data[col].apply(asignar_grupo)


  for col in ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO']:
      data[col] = data[col].apply(normalize_text)
  for col in ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO']:
      data[col] = group_values_by_frequency(data, col)

  mapeo = {'Si': 1, 'S': 1, 'No': 0, 'N':0}
  target_columns = ['FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO']
  for column in target_columns:
      data[column] = data[column].map(mapeo)
  for col in target_columns:
    data[col] = data[col].fillna(0)

  mapeo_estrato = {'Sin Estrato': 0, 'Estrato 1': 1, 'Estrato 2': 2, 'Estrato 3': 3, 'Estrato 4': 4, 'Estrato 5': 5, 'Estrato 6': 6}

  data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].map(mapeo_estrato)

  mode_value = data['FAMI_ESTRATOVIVIENDA'].mode()[0]


  data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].fillna(mode_value)

  variables_continuas = ['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA']

  mapeo_valores_matricula = {
      'Menos de 500 mil': 0,
      'Entre 500 mil y menos de 1 millón': 1,
      'Entre 1 millón y menos de 2.5 millones': 2,
      'Entre 2.5 millones y menos de 4 millones': 3,
      'Entre 4 millones y menos de 5.5 millones': 4,
      'Entre 5.5 millones y menos de 7 millones': 5,
      'Más de 7 millones': 6,
      'No pagó matrícula': 7
  }

  mapeo_valores_horas_trabajo = {
      '0': 0,
      'Menos de 10 horas': 1,
      'Entre 11 y 20 horas': 2,
      'Entre 21 y 30 horas': 3,
      'Más de 30 horas': 4
  }


  data['ESTU_VALORMATRICULAUNIVERSIDAD'] = data['ESTU_VALORMATRICULAUNIVERSIDAD'].map(mapeo_valores_matricula)
  data['ESTU_HORASSEMANATRABAJA'] = data['ESTU_HORASSEMANATRABAJA'].map(mapeo_valores_horas_trabajo)
  for col in variables_continuas:
    data[col] = data[col].fillna(0)

  moda_padre = data['FAMI_EDUCACIONPADRE'].mode()[0]
  moda_madre = data['FAMI_EDUCACIONMADRE'].mode()[0]


  data['FAMI_EDUCACIONPADRE'] = data['FAMI_EDUCACIONPADRE'].fillna(moda_padre)
  data['FAMI_EDUCACIONMADRE'] = data['FAMI_EDUCACIONMADRE'].fillna(moda_madre)
  data = pd.get_dummies(data, columns=[ 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE','ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO'], prefix=['FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE','ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO'])
  data.replace({True: 1, False: 0})

  return data


In [31]:
dtr = train
dts = test
lentr = len(dtr)
dtr.shape, dts.shape

((692500, 12), (296786, 12))

In [54]:
source_cols = [i for i in dtr.columns if i!="RENDIMIENTO_GLOBAL"]
all_data = pd.concat((dtr[source_cols], dts[source_cols]))
all_data.index = range(len(all_data))
all_data = limpiar_y_preprocesar(all_data)

Xtrk, ytrk = all_data.iloc[:lentr].values, dtr["RENDIMIENTO_GLOBAL"].values
Xtsk  = all_data.iloc[lentr:].values

print (Xtrk.shape, ytrk.shape)
print (Xtsk.shape)

<ipython-input-53-c1b6a86ec158>:94: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({True: 1, False: 0})


(692500, 36) (692500,)
(296786, 36)


In [33]:
n = int(len(Xtrk)*0.7)
n

484749

In [34]:
idxs = np.random.permutation(len(Xtrk))
idxs_trm = idxs[:n]
idxs_tsm = idxs[n:]

Xtrm = Xtrk[idxs_trm]
ytrm = ytrk[idxs_trm]

Xtsm = Xtrk[idxs_tsm]
ytsm = ytrk[idxs_tsm]

print (Xtrm.shape, ytrm.shape, Xtsm.shape, ytsm.shape)

(484749, 36) (484749,) (207751, 36) (207751,)


In [57]:
# Define model
RANDOM_SEED = 42
model = XGBClassifier(random_state=RANDOM_SEED)
# Create a LabelEncoder object
le = LabelEncoder()

# Fit the encoder to your target variable and transform it
ytrk_encoded = le.fit_transform(ytrk)
# Use cross validation to evaluate the model
scores = cross_val_score(model, Xtrk, ytrk_encoded, cv = 5, scoring = 'accuracy')
print(scores)
print(f'Mean Accuracy: {np.mean(scores)}')

[0.40498195 0.40045487 0.40311913 0.40231047 0.40092419]
Mean Accuracy: 0.4023581227436823


In [59]:
model.fit(Xtrk, ytrk_encoded)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [60]:
preds = model.predict(Xtsk)
preds

array([2, 2, 0, ..., 2, 0, 0])

array(['alto', 'alto', 'alto', ..., 'alto', 'alto', 'alto'], dtype=object)

In [61]:
real_predictions = le.inverse_transform(preds)
real_predictions

array(['medio-alto', 'medio-alto', 'alto', ..., 'medio-alto', 'alto',
       'alto'], dtype=object)

In [49]:
submission = pd.DataFrame([dts.ID, pd.Series(real_predictions, name="RENDIMIENTO_GLOBAL")]).T
submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236.0,alto
1,98545.0,alto
2,499179.0,alto
3,782980.0,alto
4,785185.0,alto


In [62]:
df_submission = pd.DataFrame({
     'ID': test.ID,
     'RENDIMIENTO_GLOBAL': real_predictions
})

df_submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,medio-alto
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo


In [63]:
df_submission.to_csv('entregadefinitiva.csv', index=False)